In [ ]:
import os

from google import genai
from google.genai import types

In [ ]:
help(types.Tool)

In [ ]:
help(types.FunctionDeclaration)

In [ ]:
help(types.ToolCodeExecution)

In [ ]:
client = genai.Client(
    api_key=os.environ["GOOGLE_API_KEY"], http_options={"api_version": "v1alpha"}
)

model_config = types.GenerateContentConfig(
    # system_instruction="""
    # """,
    safety_settings=[
        # types.SafetySetting(
        # 		category=types.HarmCategory.HARM_CATEGORY_UNSPECIFIED,
        # 		threshold=types.HarmBlockThreshold.BLOCK_NONE,
        # 		),
        # types.SafetySetting(
        # 	category=types.HarmCategory.HARM_CATEGORY_DEROGATORY,
        # 		threshold=types.HarmBlockThreshold.BLOCK_NONE,
        # 		),
        # types.SafetySetting(
        # 	category=types.HarmCategory.HARM_CATEGORY_TOXICITY,
        # 		threshold=types.HarmBlockThreshold.BLOCK_NONE,
        # 		),
        # types.SafetySetting(
        # 	category=types.HarmCategory.HARM_CATEGORY_VIOLENCE,
        # 		threshold=types.HarmBlockThreshold.BLOCK_NONE,
        # 		),
        # types.SafetySetting(
        # 	category=types.HarmCategory.HARM_CATEGORY_SEXUAL,
        # 		threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        # 		),
        # types.SafetySetting(
        # 	category=types.HarmCategory.HARM_CATEGORY_MEDICAL,
        # 		threshold=types.HarmBlockThreshold.BLOCK_NONE,
        # 		),
        # types.SafetySetting(
        # 	category=types.HarmCategory.HARM_CATEGORY_DANGEROUS,
        # 		threshold=types.HarmBlockThreshold.BLOCK_NONE,
        # 		),
        types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
            threshold=types.HarmBlockThreshold.BLOCK_NONE,
        ),
        types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            threshold=types.HarmBlockThreshold.BLOCK_NONE,
        ),
        types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        ),
        types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=types.HarmBlockThreshold.BLOCK_NONE,
        ),
        types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
            threshold=types.HarmBlockThreshold.BLOCK_NONE,
        ),
    ],
    max_output_tokens=None,
    temperature=0.8,
)

In [ ]:
# response = client.models.generate_content(
#   model='gemini-2.0-flash',
#   contents='What is the sum of the first 50 prime numbers? '
#            'Generate and run code for the calculation, and make sure you get all 50.',
#   config=types.GenerateContentConfig(
#     tools=[types.Tool(
#       code_execution=types.ToolCodeExecution
#     )]
#   )
# )

In [ ]:
model_name = "gemini-2.0-flash"  # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.0-pro-exp-02-05"] {"allow-input":true}
# model_name = "gemini-2.0-pro-exp-02-05"
# model_name = "gemini-2.0-flash-thinking-exp-01-21" #NOTE: Doesn't have tool use as an option yet


def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")


# def power_disco_ball(power: bool) -> bool:
#     """Powers the spinning disco ball."""
#     print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
#     return True


def start_music(energetic: bool, loud: bool) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True


light_controls = [enable_lights, set_light_color, stop_lights, start_music, dim_lights]
# instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color, and brightness. Do not perform any other tasks."

config = {
    # Set the available functions.
    "tools": light_controls,
    # Disable AFC so you can inspect the results first.
    "automatic_function_calling": {"disable": False},
    # Force the model to act (call 'any' function), instead of chatting.
    "tool_config": {
        "function_calling_config": {
            "mode": "AUTO"  # any #NOTE, forcing any in a chat seems to cause it to recurse and loop?
        }
    },
    # 'system_instruction':instruction,
}
chat = client.chats.create(
    model=model_name,
    # config = dict(
    #     tools=light_controls,
    #     system_instruction=instruction,
    # )
    config=model_config.model_dump() | config,
)
response = chat.send_message("Turn the lights down to a romantic level")
print(type(response.text))
if isinstance(response.text, str):
    print("len(response.text): ", len(response.text))
print(response.prompt_feedback)
print(response.text)